### Pyiceburg

* [pyiceburg documentation](https://py.iceberg.apache.org/#write-a-pyarrow-dataframe)

In [ ]:
import os

import pyarrow.parquet as pq
import pyarrow.compute as pc
from pyiceberg.catalog.sql import SqlCatalog

In [8]:
warehouse_path = "/tmp/warehouse"
os.makedirs(warehouse_path, exist_ok=True)

In [9]:
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)
namespace = "default"

In [ ]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o /tmp/yellow_tripdata_2023-01.parquet


In [ ]:
!ls /tmp/warehouse

In [ ]:
df = pq.read_table("/tmp/yellow_tripdata_2023-01.parquet")
df.schema

In [ ]:
df

In [13]:
catalog.create_namespace(namespace)
table = catalog.create_table(f"{namespace}.texi_dataset", schema=df.schema)

In [ ]:
table = catalog.load_table(f"{namespace}.texi_dataset")
table

In [ ]:
table.append(df)
len(table.scan().to_arrow())

In [19]:
df = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

In [20]:
with table.update_schema() as update_schema:
    update_schema.union_by_name(df.schema)

In [ ]:
table.overwrite(df)
print(table.scan().to_arrow())

In [ ]:
df = table.scan(row_filter="tip_per_mile > 0").to_arrow()
len(df)

In [ ]:
!find /tmp/warehouse/

In [ ]:
table